<a href="https://colab.research.google.com/github/MeTaNoV/lightFinder/blob/main/ai/training/Phi_4-Conversational.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### News

**[NEW] We've fixed many bugs in Phi-4** which greatly increases Phi-4's accuracy. See our [blogpost](https://unsloth.ai/blog/phi4)

[NEW] You can view all Phi-4 model uploads with our bug fixes including [dynamic 4-bit quants](https://unsloth.ai/blog/dynamic-4bit), GGUF & more [here](https://huggingface.co/collections/unsloth/phi-4-all-versions-677eecf93784e61afe762afa)

[NEW] As of Novemeber 2024, Unsloth now supports [vision finetuning](https://unsloth.ai/blog/vision)!


### Installation

In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
# !pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

### Unsloth

In [ ]:
from unsloth import FastLanguageModel  # FastVisionModel for LLMs
import torch
max_seq_length = 2048  # Choose any! We auto support RoPE Scaling internally!
load_in_4bit = True  # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",  # Llama-3.1 2x faster
    "unsloth/Mistral-Small-Instruct-2409",  # Mistral 22b 2x faster!
    "unsloth/Phi-4",  # Phi-4 2x faster!
    "unsloth/Phi-4-unsloth-bnb-4bit",  # Phi-4 Unsloth Dynamic 4-bit Quant
    "unsloth/gemma-2-9b-bnb-4bit",  # Gemma 2x faster!
    "unsloth/Qwen2.5-7B-Instruct-bnb-4bit"  # Qwen 2.5 2x faster!
    "unsloth/Llama-3.2-1B-bnb-4bit",  # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
]  # More models at https://docs.unsloth.ai/get-started/all-our-models

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-4",
    max_seq_length = max_seq_length,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.6: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/160k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.39G [00:00<?, ?B/s]

We now add LoRA adapters for parameter efficient finetuning - this allows us to only efficiently train 1% of all parameters.

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.1.6 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the `Phi-4` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. But we convert it to HuggingFace's normal multiturn format `("role", "content")` instead of `("from", "value")`/ Phi-4 renders multi turn conversations like below:

```
<|im_start|>user<|im_sep|>Hello!<|im_end|>
<|im_start|>assistant<|im_sep|>Hi! How can I help?<|im_end|>
<|im_start|>user<|im_sep|>What is 2+2?<|im_end|>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, phi4, llama3` and more.

In [4]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-4",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [
        tokenizer.apply_chat_template(
            convo, tokenize = False, add_generation_prompt = False
        )
        for convo in convos
    ]
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("json", data_files="./data/dataset.json", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

We now use `standardize_sharegpt` to convert ShareGPT style datasets into HuggingFace's generic format. This changes the dataset from looking like:
```
{"from": "system", "value": "You are an assistant"}
{"from": "human", "value": "What is 2+2?"}
{"from": "gpt", "value": "It's 4."}
```
to
```
{"role": "system", "content": "You are an assistant"}
{"role": "user", "content": "What is 2+2?"}
{"role": "assistant", "content": "It's 4."}
```

In [5]:
from unsloth.chat_templates import standardize_sharegpt

dataset = standardize_sharegpt(dataset)
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)

Standardizing format:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

We look at how the conversations are structured for item 5:

In [6]:
dataset[1]["conversations"]

[{'content': 'Can you fill the room area with the same light pattern in: \\n{"length": 170, "width": 149, "coordinates": [{"x": 0, "y": 0}, {"x": 0, "y": 149}, {"x": 170, "y": 149}, {"x": 170, "y": 0}], "lights": [{"x": 20.0, "y": 20.0}, {"x": 20.0, "y": 33.625}, {"x": 20.0, "y": 47.25}, {"x": 20.0, "y": 60.875}, {"x": 20.0, "y": 74.5}, {"x": 20.0, "y": 88.125}, {"x": 20.0, "y": 101.75}, {"x": 20.0, "y": 115.375}, {"x": 20.0, "y": 129.0}, {"x": 34.44444444444444, "y": 20.0}, {"x": 34.44444444444444, "y": 129.0}]}',
  'role': 'user'},
 {'content': '{"length": 170, "width": 149, "coordinates": [{"x": 0, "y": 0}, {"x": 0, "y": 149}, {"x": 170, "y": 149}, {"x": 170, "y": 0}], "lights": [{"x": 20.0, "y": 20.0}, {"x": 20.0, "y": 33.625}, {"x": 20.0, "y": 47.25}, {"x": 20.0, "y": 60.875}, {"x": 20.0, "y": 74.5}, {"x": 20.0, "y": 88.125}, {"x": 20.0, "y": 101.75}, {"x": 20.0, "y": 115.375}, {"x": 20.0, "y": 129.0}, {"x": 34.44444444444444, "y": 20.0}, {"x": 34.44444444444444, "y": 129.0}, {"x"

And we see how the chat template transformed these conversations.

In [7]:
dataset[1]["text"]

'<|im_start|>user<|im_sep|>Can you fill the room area with the same light pattern in: \\n{"length": 170, "width": 149, "coordinates": [{"x": 0, "y": 0}, {"x": 0, "y": 149}, {"x": 170, "y": 149}, {"x": 170, "y": 0}], "lights": [{"x": 20.0, "y": 20.0}, {"x": 20.0, "y": 33.625}, {"x": 20.0, "y": 47.25}, {"x": 20.0, "y": 60.875}, {"x": 20.0, "y": 74.5}, {"x": 20.0, "y": 88.125}, {"x": 20.0, "y": 101.75}, {"x": 20.0, "y": 115.375}, {"x": 20.0, "y": 129.0}, {"x": 34.44444444444444, "y": 20.0}, {"x": 34.44444444444444, "y": 129.0}]}<|im_end|><|im_start|>assistant<|im_sep|>{"length": 170, "width": 149, "coordinates": [{"x": 0, "y": 0}, {"x": 0, "y": 149}, {"x": 170, "y": 149}, {"x": 170, "y": 0}], "lights": [{"x": 20.0, "y": 20.0}, {"x": 20.0, "y": 33.625}, {"x": 20.0, "y": 47.25}, {"x": 20.0, "y": 60.875}, {"x": 20.0, "y": 74.5}, {"x": 20.0, "y": 88.125}, {"x": 20.0, "y": 101.75}, {"x": 20.0, "y": 115.375}, {"x": 20.0, "y": 129.0}, {"x": 34.44444444444444, "y": 20.0}, {"x": 34.44444444444444,

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        # max_steps = 30,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/500 [00:00<?, ? examples/s]

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [9]:
from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
    trainer,
    instruction_part="<|im_start|>user<|im_sep|>",
    response_part="<|im_start|>assistant<|im_sep|>",
)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

We verify masking is actually done:

In [10]:
tokenizer.decode(trainer.train_dataset[1]["input_ids"])

'<|im_start|>user<|im_sep|>Can you fill the room area with the same light pattern in: \\n{"length": 170, "width": 149, "coordinates": [{"x": 0, "y": 0}, {"x": 0, "y": 149}, {"x": 170, "y": 149}, {"x": 170, "y": 0}], "lights": [{"x": 20.0, "y": 20.0}, {"x": 20.0, "y": 33.625}, {"x": 20.0, "y": 47.25}, {"x": 20.0, "y": 60.875}, {"x": 20.0, "y": 74.5}, {"x": 20.0, "y": 88.125}, {"x": 20.0, "y": 101.75}, {"x": 20.0, "y": 115.375}, {"x": 20.0, "y": 129.0}, {"x": 34.44444444444444, "y": 20.0}, {"x": 34.44444444444444, "y": 129.0}]}<|im_end|><|im_start|>assistant<|im_sep|>{"length": 170, "width": 149, "coordinates": [{"x": 0, "y": 0}, {"x": 0, "y": 149}, {"x": 170, "y": 149}, {"x": 170, "y": 0}], "lights": [{"x": 20.0, "y": 20.0}, {"x": 20.0, "y": 33.625}, {"x": 20.0, "y": 47.25}, {"x": 20.0, "y": 60.875}, {"x": 20.0, "y": 74.5}, {"x": 20.0, "y": 88.125}, {"x": 20.0, "y": 101.75}, {"x": 20.0, "y": 115.375}, {"x": 20.0, "y": 129.0}, {"x": 34.44444444444444, "y": 20.0}, {"x": 34.44444444444444,

In [11]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[1]["labels"]])

'                                                                                                                                                                                                                                                                                 {"length": 170, "width": 149, "coordinates": [{"x": 0, "y": 0}, {"x": 0, "y": 149}, {"x": 170, "y": 149}, {"x": 170, "y": 0}], "lights": [{"x": 20.0, "y": 20.0}, {"x": 20.0, "y": 33.625}, {"x": 20.0, "y": 47.25}, {"x": 20.0, "y": 60.875}, {"x": 20.0, "y": 74.5}, {"x": 20.0, "y": 88.125}, {"x": 20.0, "y": 101.75}, {"x": 20.0, "y": 115.375}, {"x": 20.0, "y": 129.0}, {"x": 34.44444444444444, "y": 20.0}, {"x": 34.44444444444444, "y": 129.0}, {"x": 48.888888888888886, "y": 20.0}, {"x": 48.888888888888886, "y": 129.0}, {"x": 63.333333333333336, "y": 20.0}, {"x": 63.333333333333336, "y": 129.0}, {"x": 77.77777777777777, "y": 20.0}, {"x": 77.77777777777777, "y": 129.0}, {"x": 92.22222222222223, "y": 20.0}, {"x": 92.222222222

We can see the System and Instruction prompts are successfully masked!

In [12]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.564 GB.
10.043 GB of memory reserved.


In [13]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 500 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 62
 "-____-"     Number of trainable parameters = 65,536,000


Step,Training Loss
1,0.039000
2,0.038200
3,0.038800
4,0.031800
5,0.023500
6,0.020200
7,0.005200
8,0.009100
9,0.007700
10,0.005300


In [14]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

506.0812 seconds used for training.
8.43 minutes used for training.
Peak reserved memory = 11.713 GB.
Peak reserved memory for training = 1.67 GB.
Peak reserved memory % of max memory = 29.605 %.
Peak reserved memory for training % of max memory = 4.221 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

We use `min_p = 0.1` and `temperature = 1.5`.

In [15]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-4",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(100352, 5120, padding_idx=100351)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=5120, out_features=5120, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=5120, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

In [16]:
messages = [
    {"role": "user", "content": "Can you fill the room area with the same light pattern in: \\n{\"length\": 157, \"width\": 103, \"coordinates\": [{\"x\": 0, \"y\": 0}, {\"x\": 0, \"y\": 103}, {\"x\": 157, \"y\": 103}, {\"x\": 157, \"y\": 0}], \"lights\": [{\"x\": 20.0, \"y\": 20.0}, {\"x\": 20.0, \"y\": 36.6}, {\"x\": 37.125, \"y\": 20.0}, {\"x\": 139.875, \"y\": 20.0}, {\"x\": 139.875, \"y\": 36.6}]}"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(
    input_ids = inputs, max_new_tokens = 64, use_cache = True, temperature = 0.0, min_p = 0.95
)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|im_start|>user<|im_sep|>Can you fill the room area with the same light pattern in: \\n{"length": 157, "width": 103, "coordinates": [{"x": 0, "y": 0}, {"x": 0, "y": 103}, {"x": 157, "y": 103}, {"x": 157, "y": 0}], "lights": [{"x": 20.0, "y": 20.0}, {"x": 20.0, "y": 36.6}, {"x": 37.125, "y": 20.0}, {"x": 139.875, "y": 20.0}, {"x": 139.875, "y": 36.6}]}<|im_end|><|im_start|>assistant<|im_sep|>{"length": 157, "width": 103, "coordinates": [{"x": 0, "y": 0}, {"x": 0, "y": 103}, {"x": 157, "y": 103}, {"x": 157, "y": 0}], "']

In [17]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-4",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Can you fill the room area with the same light pattern in: \\n{\"length\": 107, \"width\": 87, \"coordinates\": [{\"x\": 0, \"y\": 0}, {\"x\": 0, \"y\": 87}, {\"x\": 107, \"y\": 87}, {\"x\": 107, \"y\": 0}], \"lights\": [{\"x\": 20.0, \"y\": 20.0}, {\"x\": 20.0, \"y\": 31.75}, {\"x\": 20.0, \"y\": 43.5}, {\"x\": 20.0, \"y\": 55.25}, {\"x\": 20.0, \"y\": 67.0}, {\"x\": 33.4, \"y\": 20.0}, {\"x\": 33.4, \"y\": 67.0}]}"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(
    input_ids = inputs, max_new_tokens = 2048, use_cache = True, temperature = 0.0, min_p = 0.95
)
tokenizer.batch_decode(outputs)

['<|im_start|>user<|im_sep|>Can you fill the room area with the same light pattern in: \\n{"length": 107, "width": 87, "coordinates": [{"x": 0, "y": 0}, {"x": 0, "y": 87}, {"x": 107, "y": 87}, {"x": 107, "y": 0}], "lights": [{"x": 20.0, "y": 20.0}, {"x": 20.0, "y": 31.75}, {"x": 20.0, "y": 43.5}, {"x": 20.0, "y": 55.25}, {"x": 20.0, "y": 67.0}, {"x": 33.4, "y": 20.0}, {"x": 33.4, "y": 67.0}]}<|im_end|><|im_start|>assistant<|im_sep|>{"length": 107, "width": 87, "coordinates": [{"x": 0, "y": 0}, {"x": 0, "y": 87}, {"x": 107, "y": 87}, {"x": 107, "y": 0}], "lights": [{"x": 20.0, "y": 20.0}, {"x": 20.0, "y": 31.75}, {"x": 20.0, "y": 43.5}, {"x": 20.0, "y": 55.25}, {"x": 20.0, "y": 67.0}, {"x": 33.4, "y": 20.0}, {"x": 33.4, "y": 67.0}, {"x": 46.8, "y": 20.0}, {"x": 46.8, "y": 67.0}, {"x": 60.2, "y": 20.0}, {"x": 60.2, "y": 67.0}, {"x": 73.6, "y": 20.0}, {"x": 73.6, "y": 67.0}, {"x": 87.0, "y": 20.0}, {"x": 87.0, "y": 31.75}, {"x": 87.0, "y": 43.5}, {"x": 87.0, "y": 55.25}, {"x": 87.0, "y": 

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [18]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Can you fill the room area with the same light pattern in: \\n{\"length\": 176, \"width\": 146, \"coordinates\": [{\"x\": 0, \"y\": 0}, {\"x\": 0, \"y\": 146}, {\"x\": 176, \"y\": 146}, {\"x\": 176, \"y\": 0}], \"lights\": [{\"x\": 20.0, \"y\": 20.0}, {\"x\": 20.0, \"y\": 33.25}, {\"x\": 20.0, \"y\": 46.5}, {\"x\": 20.0, \"y\": 59.75}, {\"x\": 20.0, \"y\": 73.0}, {\"x\": 20.0, \"y\": 86.25}, {\"x\": 20.0, \"y\": 99.5}, {\"x\": 20.0, \"y\": 112.75}, {\"x\": 20.0, \"y\": 126.0}, {\"x\": 33.6, \"y\": 20.0}, {\"x\": 33.6, \"y\": 126.0}]}"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(
    input_ids = inputs, streamer = text_streamer, max_new_tokens = 1024,
    use_cache = True, temperature = 0.0, min_p = 0.95
)

{"length": 176, "width": 146, "coordinates": [{"x": 0, "y": 0}, {"x": 0, "y": 146}, {"x": 176, "y": 146}, {"x": 176, "y": 0}], "lights": [{"x": 20.0, "y": 20.0}, {"x": 20.0, "y": 33.25}, {"x": 20.0, "y": 46.5}, {"x": 20.0, "y": 59.75}, {"x": 20.0, "y": 73.0}, {"x": 20.0, "y": 86.25}, {"x": 20.0, "y": 99.5}, {"x": 20.0, "y": 112.75}, {"x": 20.0, "y": 126.0}, {"x": 33.6, "y": 20.0}, {"x": 33.6, "y": 126.0}, {"x": 47.2, "y": 20.0}, {"x": 47.2, "y": 126.0}, {"x": 60.8, "y": 20.0}, {"x": 60.8, "y": 126.0}, {"x": 74.4, "y": 20.0}, {"x": 74.4, "y": 126.0}, {"x": 88.0, "y": 20.0}, {"x": 88.0, "y": 126.0}, {"x": 101.6, "y": 20.0}, {"x": 101.6, "y": 126.0}, {"x": 115.2, "y": 20.0}, {"x": 115.2, "y": 126.0}, {"x": 128.8, "y": 20.0}, {"x": 128.8, "y": 126.0}, {"x": 142.4, "y": 20.0}, {"x": 142.4, "y": 126.0}, {"x": 156.0, "y": 20.0}, {"x": 156.0, "y": 33.25}, {"x": 156.0, "y": 46.5}, {"x": 156.0, "y": 59.75}, {"x": 156.0, "y": 73.0}, {"x": 156.0, "y": 86.25}, {"x": 156.0, "y": 99.5}, {"x": 156.0, 

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [19]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/vocab.json',
 'lora_model/merges.txt',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Can you fill the room area with the same light pattern in: \\n{\"length\": 176, \"width\": 146, \"coordinates\": [{\"x\": 0, \"y\": 0}, {\"x\": 0, \"y\": 146}, {\"x\": 176, \"y\": 146}, {\"x\": 176, \"y\": 0}], \"lights\": [{\"x\": 20.0, \"y\": 20.0}, {\"x\": 20.0, \"y\": 33.25}, {\"x\": 20.0, \"y\": 46.5}, {\"x\": 20.0, \"y\": 59.75}, {\"x\": 20.0, \"y\": 73.0}, {\"x\": 20.0, \"y\": 86.25}, {\"x\": 20.0, \"y\": 99.5}, {\"x\": 20.0, \"y\": 112.75}, {\"x\": 20.0, \"y\": 126.0}, {\"x\": 33.6, \"y\": 20.0}, {\"x\": 33.6, \"y\": 126.0}]}"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(
    input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
    use_cache = True, temperature = 0.0, min_p = 0.95
)

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [21]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer

    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model",  # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit=load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [22]:
from google.colab import userdata
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list in our [Docs](https://docs.unsloth.ai/basics/saving-and-using-models/saving-to-gguf)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [23]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Llama 3.2 Conversational notebook. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
